In [14]:
#Import Package
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import preprocessing

#importing datasets  
df= pd.read_csv('data.csv') 

#create a dummy basis on whether the income is greater then networth
df['Financially_In_Dept']=np.where(df["TOTAL_INCOME"]>df["TOTAL_NETWORTH"],True,False)

#average gifting amount of an individual
df['Avg_Gift_Amount']= df["TOTAL_GIFT_AMOUNT"] / df["TOTAL_NUMBER_GIFTS"]

#what capacity an individual has used from their giving capacity
df['gifting_Capcity']= df["TOTAL_GIFT_AMOUNT"] / df["TOTAL_GIVING_CAPACITY"]

#Check if largest gift amount is greater than total gift amount
df['gift_check']=np.where(df["LARGEST_GIFT_AMOUNT"]>df["TOTAL_GIFT_AMOUNT"],True,False)


df=df.loc[df['gift_check'] == False]
df


,ID,LARGEST_GIFT_AMOUNT,BIRTH_DATE,FIRST_GIFT_DATE,LARGEST_GIFT_DATE,LAST_GIFT_DATE,TOTAL_GIFT_AMOUNT,TOTAL_NUMBER_GIFTS,TOTAL_GIVING_CAPACITY,TOTAL_INCOME,...,GENDER,FIRST_GIFT_AMOUNT,LAST_GIFT_AMOUNT,EDUCATION_SCHOOL_NAME,GRADUATE_DATE,TOTAL_YEARS_GIVING,Financially_In_Dept,Avg_Gift_Amount,gifting_Capcity,gift_check
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,False
1,2,NaN,12/1/1957,NaN,NaN,NaN,NaN,NaN,138967,187500,...,M,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,False
2,3,NaN,12/1/1965,NaN,NaN,NaN,NaN,NaN,15929,37500,...,M,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,False
3,4,135.0,6/1/1977,3/1/2011,3/1/2011,3/1/2011,135.0,1.0,38055,47500,...,M,135.0,135.0,NaN,NaN,135.0,False,135.0,0.003547,False
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26909,26910,NaN,6/1/1948,NaN,NaN,NaN,NaN,NaN,86751,125000,...,F,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,False
26910,26911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,M,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,False
26911,26912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,M,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,False
26912,26913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,M,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,False


In [15]:
#Convert bateofbirth from string to date and then calculate age
df['BIRTH_DATE']= pd.to_datetime(df['BIRTH_DATE'])

#age function
from datetime import date
 
def calculateAge(birthDate):
    today = date.today()
    age = today.year - birthDate.year - ((today.month, today.day) < (birthDate.month, birthDate.day))
 
    return age

#Append the age in the data frame
df['age']=df['BIRTH_DATE'].apply(calculateAge)

#Drop negative age as it is inappropriate
ind=df[df['age']<0]['ID']
df = df.drop(ind-1, axis='rows')


#Drop Unimportant column basis on the column data
df=df.drop(['EDUCATION_SCHOOL_NAME','GRADUATE_DATE','TOTAL_YEARS_GIVING'], axis=1)

#Drop rows with NA
df = df.dropna()

#Converting all the dates into dataframe actual date format after that selected years from it so that we can use them in clustering 
df['FIRST_GIFT_DATE']= pd.to_datetime(df['FIRST_GIFT_DATE']).dt.year
df['LARGEST_GIFT_DATE']= pd.to_datetime(df['LARGEST_GIFT_DATE']).dt.year
df['LAST_GIFT_DATE']= pd.to_datetime(df['LAST_GIFT_DATE']).dt.year
df

,ID,LARGEST_GIFT_AMOUNT,BIRTH_DATE,FIRST_GIFT_DATE,LARGEST_GIFT_DATE,LAST_GIFT_DATE,TOTAL_GIFT_AMOUNT,TOTAL_NUMBER_GIFTS,TOTAL_GIVING_CAPACITY,TOTAL_INCOME,...,LARGEST_REALEST_VALUE,TOTAL_NO_EVENTS_PARTICIPATED,GENDER,FIRST_GIFT_AMOUNT,LAST_GIFT_AMOUNT,Financially_In_Dept,Avg_Gift_Amount,gifting_Capcity,gift_check,age
3,4,135.0,1977-06-01,2011,2011,2011,135.0,1.0,38055,47500,...,539909,0.0,M,135.0,135.0,False,135.000000,0.003547,False,45.0
19,20,2500.0,1959-01-01,2021,2021,2021,2500.0,1.0,17400819,2802980,...,2393990,1.0,M,2500.0,2500.0,False,2500.000000,0.000144,False,64.0
20,21,50.0,1957-12-01,2012,2012,2012,50.0,1.0,90861,187500,...,241225,0.0,M,50.0,50.0,False,50.000000,0.000550,False,65.0
23,24,7200.0,1938-11-23,2000,2006,2006,18200.0,3.0,260330,375000,...,267601,0.0,M,6000.0,5400.0,False,6066.666667,0.069911,False,84.0
27,28,240.0,1946-10-01,2000,2004,2009,1010.0,7.0,54790,71000,...,278708,0.0,M,144.0,48.0,False,144.285714,0.018434,False,76.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26834,26835,600.0,1983-04-01,2006,2006,2006,1200.0,2.0,29490,125000,...,140790,0.0,M,600.0,600.0,False,600.000000,0.040692,False,40.0
26860,26861,180.0,1981-08-01,2011,2011,2011,180.0,1.0,14050,73000,...,0,0.0,F,180.0,180.0,True,180.000000,0.012811,False,41.0
26863,26864,180.0,1961-12-01,2010,2014,2014,855.0,6.0,85872,87500,...,711326,0.0,M,180.0,15.0,False,142.500000,0.009957,False,61.0
26877,26878,50.0,1940-04-01,2015,2015,2015,50.0,1.0,159362,225000,...,486230,0.0,F,50.0,50.0,False,50.000000,0.000314,False,83.0


In [16]:
df.dropna(axis=1)

,ID,LARGEST_GIFT_AMOUNT,BIRTH_DATE,FIRST_GIFT_DATE,LARGEST_GIFT_DATE,LAST_GIFT_DATE,TOTAL_GIFT_AMOUNT,TOTAL_NUMBER_GIFTS,TOTAL_GIVING_CAPACITY,TOTAL_INCOME,...,LARGEST_REALEST_VALUE,TOTAL_NO_EVENTS_PARTICIPATED,GENDER,FIRST_GIFT_AMOUNT,LAST_GIFT_AMOUNT,Financially_In_Dept,Avg_Gift_Amount,gifting_Capcity,gift_check,age
3,4,135.0,1977-06-01,2011,2011,2011,135.0,1.0,38055,47500,...,539909,0.0,M,135.0,135.0,False,135.000000,0.003547,False,45.0
19,20,2500.0,1959-01-01,2021,2021,2021,2500.0,1.0,17400819,2802980,...,2393990,1.0,M,2500.0,2500.0,False,2500.000000,0.000144,False,64.0
20,21,50.0,1957-12-01,2012,2012,2012,50.0,1.0,90861,187500,...,241225,0.0,M,50.0,50.0,False,50.000000,0.000550,False,65.0
23,24,7200.0,1938-11-23,2000,2006,2006,18200.0,3.0,260330,375000,...,267601,0.0,M,6000.0,5400.0,False,6066.666667,0.069911,False,84.0
27,28,240.0,1946-10-01,2000,2004,2009,1010.0,7.0,54790,71000,...,278708,0.0,M,144.0,48.0,False,144.285714,0.018434,False,76.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26834,26835,600.0,1983-04-01,2006,2006,2006,1200.0,2.0,29490,125000,...,140790,0.0,M,600.0,600.0,False,600.000000,0.040692,False,40.0
26860,26861,180.0,1981-08-01,2011,2011,2011,180.0,1.0,14050,73000,...,0,0.0,F,180.0,180.0,True,180.000000,0.012811,False,41.0
26863,26864,180.0,1961-12-01,2010,2014,2014,855.0,6.0,85872,87500,...,711326,0.0,M,180.0,15.0,False,142.500000,0.009957,False,61.0
26877,26878,50.0,1940-04-01,2015,2015,2015,50.0,1.0,159362,225000,...,486230,0.0,F,50.0,50.0,False,50.000000,0.000314,False,83.0


In [11]:
df1=df.append(df)
df1.drop_duplicates(inplace=True,keep=False)
df1

C:\Users\rathakra2201\AppData\Local\Temp\1\ipykernel_21564\1950072000.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1=df.append(df)


,ID,LARGEST_GIFT_AMOUNT,BIRTH_DATE,FIRST_GIFT_DATE,LARGEST_GIFT_DATE,LAST_GIFT_DATE,TOTAL_GIFT_AMOUNT,TOTAL_NUMBER_GIFTS,TOTAL_GIVING_CAPACITY,TOTAL_INCOME,...,LARGEST_REALEST_VALUE,TOTAL_NO_EVENTS_PARTICIPATED,GENDER,FIRST_GIFT_AMOUNT,LAST_GIFT_AMOUNT,Financially_In_Dept,Avg_Gift_Amount,gifting_Capcity,gift_check,age


In [18]:
#dropping all the records where Average gift amount lies between 500$ to 5000$ becuase it is already a group
df.drop(df[ (df['Avg_Gift_Amount'] >= 500) & (df['Avg_Gift_Amount'] <= 5000)].index, inplace = True)


In [19]:
#Correlation Matrix
df.corr(method='pearson', min_periods=1)

,ID,LARGEST_GIFT_AMOUNT,FIRST_GIFT_DATE,LARGEST_GIFT_DATE,LAST_GIFT_DATE,TOTAL_GIFT_AMOUNT,TOTAL_NUMBER_GIFTS,TOTAL_GIVING_CAPACITY,TOTAL_INCOME,TOTAL_NETWORTH,...,TOTAL_EAD,LARGEST_REALEST_VALUE,TOTAL_NO_EVENTS_PARTICIPATED,FIRST_GIFT_AMOUNT,LAST_GIFT_AMOUNT,Financially_In_Dept,Avg_Gift_Amount,gifting_Capcity,gift_check,age
ID,1.000000,0.022614,0.007959,0.026087,0.008678,0.029664,-0.011250,0.021329,-0.019445,-0.001476,...,-0.006843,0.008229,0.016435,0.011441,0.028955,-0.001950,0.020059,0.011496,NaN,-0.011730
LARGEST_GIFT_AMOUNT,0.022614,1.000000,0.007692,0.006492,0.022674,0.865750,0.021613,0.761532,0.013084,0.091815,...,0.076721,0.055468,0.090446,0.769712,0.729769,-0.018194,0.911364,0.467235,NaN,0.013406
FIRST_GIFT_DATE,0.007959,0.007692,1.000000,0.593429,0.330132,0.000158,-0.333122,-0.039750,-0.050918,-0.052779,...,-0.059636,0.010449,-0.106518,0.018239,0.012261,0.031840,0.018718,-0.094375,NaN,-0.152410
LARGEST_GIFT_DATE,0.026087,0.006492,0.593429,1.000000,0.657309,-0.005215,-0.087354,-0.035817,-0.040408,-0.036263,...,-0.047519,0.032915,0.064529,0.005154,0.006136,-0.051740,0.008524,-0.029138,NaN,-0.083257
LAST_GIFT_DATE,0.008678,0.022674,0.330132,0.657309,1.000000,0.016755,0.209804,-0.002083,-0.036375,-0.015858,...,-0.021258,0.059853,0.185971,0.012999,0.005995,-0.066432,0.015411,0.054668,NaN,-0.043500
TOTAL_GIFT_AMOUNT,0.029664,0.865750,0.000158,-0.005215,0.016755,1.000000,0.022294,0.854898,-0.000374,0.059314,...,0.048519,0.011792,0.042916,0.895485,0.592939,-0.011907,0.929736,0.360843,NaN,0.027050
TOTAL_NUMBER_GIFTS,-0.011250,0.021613,-0.333122,-0.087354,0.209804,0.022294,1.000000,0.014304,0.000845,-0.011778,...,0.005226,-0.028024,0.281488,0.001045,0.001417,-0.027511,-0.000785,0.293502,NaN,0.025780
TOTAL_GIVING_CAPACITY,0.021329,0.761532,-0.039750,-0.035817,-0.002083,0.854898,0.014304,1.000000,0.369314,0.511816,...,0.488064,0.293556,0.073964,0.773153,0.516753,-0.149992,0.806476,0.273146,NaN,0.131648
TOTAL_INCOME,-0.019445,0.013084,-0.050918,-0.040408,-0.036375,-0.000374,0.000845,0.369314,1.000000,0.508583,...,0.638105,0.325412,0.095996,-0.012313,-0.002586,-0.237022,-0.005590,-0.093087,NaN,0.033062
TOTAL_NETWORTH,-0.001476,0.091815,-0.052779,-0.036263,-0.015858,0.059314,-0.011778,0.511816,0.508583,1.000000,...,0.907373,0.546525,0.087261,0.043419,0.043234,-0.220744,0.059449,-0.013349,NaN,0.171662


In [20]:
#Drop basis on high correlation and create a new database, also remove birthdate as we have age
newdf1=df.drop(['TOTAL_GIFT_AMOUNT','TOTAL_NETWORTH','FIRST_GIFT_AMOUNT','BIRTH_DATE'], axis=1)

#Drop ID column
newdf1=newdf1.drop(['ID'], axis=1)

#updating all the numeric columns to float from string
newdf1 = newdf1.astype({'LARGEST_GIFT_AMOUNT': float,'TOTAL_NUMBER_GIFTS': float,'TOTAL_GIVING_CAPACITY': float,'TOTAL_INCOME': float,'TOTAL_DONATIONS': float,'TOTAL_EAD': float,'LARGEST_REALEST_VALUE': float,'LAST_GIFT_AMOUNT': float,'age': float,'Avg_Gift_Amount': float,'gifting_Capcity': float})


#Scaling the numeric columns
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
newdf1[['LARGEST_GIFT_AMOUNT', 'TOTAL_NUMBER_GIFTS', 'TOTAL_GIVING_CAPACITY', 'TOTAL_INCOME', 'TOTAL_DONATIONS', 'TOTAL_EAD', 'LARGEST_REALEST_VALUE', 'LAST_GIFT_AMOUNT', 'age', 'Avg_Gift_Amount', 'gifting_Capcity']] = scaler.fit_transform(newdf1[['LARGEST_GIFT_AMOUNT', 'TOTAL_NUMBER_GIFTS', 'TOTAL_GIVING_CAPACITY', 'TOTAL_INCOME', 'TOTAL_DONATIONS', 'TOTAL_EAD', 'LARGEST_REALEST_VALUE','LAST_GIFT_AMOUNT', 'age', 'Avg_Gift_Amount', 'gifting_Capcity']])
newdf1

,LARGEST_GIFT_AMOUNT,FIRST_GIFT_DATE,LARGEST_GIFT_DATE,LAST_GIFT_DATE,TOTAL_NUMBER_GIFTS,TOTAL_GIVING_CAPACITY,TOTAL_INCOME,TOTAL_DONATIONS,TOTAL_EAD,LARGEST_REALEST_VALUE,TOTAL_NO_EVENTS_PARTICIPATED,GENDER,LAST_GIFT_AMOUNT,Financially_In_Dept,Avg_Gift_Amount,gifting_Capcity,gift_check,age
3,0.000076,2011,2011,2011,0.000000,0.005905,0.020903,0.000013,0.015884,0.182655,0.0,M,0.000228,False,0.000121,0.003451,False,0.424528
20,0.000026,2012,2012,2012,0.000000,0.014121,0.082510,0.000000,0.033019,0.081608,0.0,M,0.000085,False,0.000041,0.000453,False,0.613208
23,0.004232,2000,2006,2006,0.007519,0.040487,0.165021,0.002836,0.117621,0.090531,0.0,M,0.009134,False,0.005682,0.069821,False,0.792453
27,0.000137,2000,2004,2009,0.022556,0.008509,0.031244,0.000496,0.024266,0.094289,0.0,M,0.000081,False,0.000129,0.018339,False,0.716981
35,0.000114,2009,2010,2013,0.011278,0.032169,0.055007,0.000095,0.048986,0.353934,0.0,F,0.000304,False,0.000149,0.003094,False,0.396226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26827,0.000114,1991,2013,2015,0.018797,0.007656,0.019803,0.000078,0.023249,0.116782,0.0,M,0.000169,False,0.000080,0.011058,False,0.877358
26860,0.000102,2011,2011,2011,0.000000,0.002170,0.032124,0.000020,0.018344,0.000000,0.0,F,0.000304,True,0.000163,0.012716,False,0.386792
26863,0.000102,2010,2014,2014,0.018797,0.013345,0.038505,0.000126,0.041442,0.240647,0.0,M,0.000025,False,0.000128,0.009861,False,0.575472
26877,0.000026,2015,2015,2015,0.000000,0.024778,0.099013,0.000000,0.084602,0.164495,0.0,F,0.000085,False,0.000041,0.000217,False,0.783019


In [21]:
#Apply K-Mode clustering(which is used when there is both numeric and category data)
from kmodes.kprototypes import KPrototypes
kproto = KPrototypes(n_clusters=9, init='Cao')
clusters = kproto.fit_predict(newdf1, categorical=[1,2,3,10,11,13])

cluster_dict=[]
for c in clusters:
    cluster_dict.append(c)
    
cluster_dict


df['Group']=cluster_dict
df

,ID,LARGEST_GIFT_AMOUNT,BIRTH_DATE,FIRST_GIFT_DATE,LARGEST_GIFT_DATE,LAST_GIFT_DATE,TOTAL_GIFT_AMOUNT,TOTAL_NUMBER_GIFTS,TOTAL_GIVING_CAPACITY,TOTAL_INCOME,...,TOTAL_NO_EVENTS_PARTICIPATED,GENDER,FIRST_GIFT_AMOUNT,LAST_GIFT_AMOUNT,Financially_In_Dept,Avg_Gift_Amount,gifting_Capcity,gift_check,age,Group
3,4,135.0,1977-06-01,2011,2011,2011,135.0,1.0,38055,47500,...,0.0,M,135.0,135.0,False,135.000000,0.003547,False,45.0,0
20,21,50.0,1957-12-01,2012,2012,2012,50.0,1.0,90861,187500,...,0.0,M,50.0,50.0,False,50.000000,0.000550,False,65.0,4
23,24,7200.0,1938-11-23,2000,2006,2006,18200.0,3.0,260330,375000,...,0.0,M,6000.0,5400.0,False,6066.666667,0.069911,False,84.0,8
27,28,240.0,1946-10-01,2000,2004,2009,1010.0,7.0,54790,71000,...,0.0,M,144.0,48.0,False,144.285714,0.018434,False,76.0,8
35,36,200.0,1980-12-01,2009,2010,2013,660.0,4.0,206862,125000,...,0.0,F,180.0,180.0,False,165.000000,0.003191,False,42.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26827,26828,200.0,1930-02-01,1991,2013,2015,550.0,6.0,49310,45000,...,0.0,M,50.0,100.0,False,91.666667,0.011154,False,93.0,8
26860,26861,180.0,1981-08-01,2011,2011,2011,180.0,1.0,14050,73000,...,0.0,F,180.0,180.0,True,180.000000,0.012811,False,41.0,0
26863,26864,180.0,1961-12-01,2010,2014,2014,855.0,6.0,85872,87500,...,0.0,M,180.0,15.0,False,142.500000,0.009957,False,61.0,3
26877,26878,50.0,1940-04-01,2015,2015,2015,50.0,1.0,159362,225000,...,0.0,F,50.0,50.0,False,50.000000,0.000314,False,83.0,7


In [22]:
df_last

,ID,LARGEST_GIFT_AMOUNT,BIRTH_DATE,FIRST_GIFT_DATE,LARGEST_GIFT_DATE,LAST_GIFT_DATE,TOTAL_GIFT_AMOUNT,TOTAL_NUMBER_GIFTS,TOTAL_GIVING_CAPACITY,TOTAL_INCOME,...,TOTAL_NO_EVENTS_PARTICIPATED,GENDER,FIRST_GIFT_AMOUNT,LAST_GIFT_AMOUNT,Financially_In_Dept,Avg_Gift_Amount,gifting_Capcity,gift_check,Group,age
19,20,2500.0,1959-01-01,2021,2021,2021,2500.0,1.0,17400819,2802980,...,1.0,M,2500.0,2500.0,False,2500.000000,0.000144,False,9,64.0
122,123,1000.0,1948-10-01,2006,2006,2006,1000.0,1.0,304460,187500,...,0.0,M,1000.0,1000.0,False,1000.000000,0.003285,False,9,74.0
132,133,600.0,1948-01-01,2004,2009,2009,1200.0,2.0,254429,375000,...,0.0,M,600.0,600.0,False,600.000000,0.004716,False,9,75.0
136,137,600.0,1960-01-01,2011,2011,2011,600.0,1.0,93360,17500,...,0.0,M,600.0,600.0,False,600.000000,0.006427,False,9,63.0
143,144,3000.0,1955-01-15,1993,2011,2020,6920.0,12.0,222723,375000,...,5.0,M,170.0,100.0,False,576.666667,0.031070,False,9,68.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26650,26651,605.0,1982-11-01,2014,2014,2014,605.0,1.0,12505,70000,...,0.0,F,605.0,605.0,True,605.000000,0.048381,False,9,40.0
26683,26684,600.0,1940-08-01,2005,2012,2012,1200.0,2.0,62147,87500,...,0.0,M,600.0,600.0,False,600.000000,0.019309,False,9,82.0
26740,26741,5000.0,1941-08-01,2020,2021,2021,10000.0,2.0,321380,187500,...,0.0,F,5000.0,5000.0,False,5000.000000,0.031116,False,9,81.0
26783,26784,20000.0,1965-09-20,1995,2012,2020,57692.0,22.0,57692,0,...,1.0,F,40.0,250.0,False,2622.363636,1.000000,False,9,57.0


In [23]:
df_final = df.append(df_last, ignore_index=True)

C:\Users\rathakra2201\AppData\Local\Temp\1\ipykernel_14152\1569060653.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df.append(df_last, ignore_index=True)


In [25]:
df_final.to_csv('final_grouping.csv', header=True, index=False)

In [24]:
df_final

,ID,LARGEST_GIFT_AMOUNT,BIRTH_DATE,FIRST_GIFT_DATE,LARGEST_GIFT_DATE,LAST_GIFT_DATE,TOTAL_GIFT_AMOUNT,TOTAL_NUMBER_GIFTS,TOTAL_GIVING_CAPACITY,TOTAL_INCOME,...,TOTAL_NO_EVENTS_PARTICIPATED,GENDER,FIRST_GIFT_AMOUNT,LAST_GIFT_AMOUNT,Financially_In_Dept,Avg_Gift_Amount,gifting_Capcity,gift_check,age,Group
0,4,135.0,1977-06-01,2011,2011,2011,135.0,1.0,38055,47500,...,0.0,M,135.0,135.0,False,135.000000,0.003547,False,45.0,0
1,21,50.0,1957-12-01,2012,2012,2012,50.0,1.0,90861,187500,...,0.0,M,50.0,50.0,False,50.000000,0.000550,False,65.0,4
2,24,7200.0,1938-11-23,2000,2006,2006,18200.0,3.0,260330,375000,...,0.0,M,6000.0,5400.0,False,6066.666667,0.069911,False,84.0,8
3,28,240.0,1946-10-01,2000,2004,2009,1010.0,7.0,54790,71000,...,0.0,M,144.0,48.0,False,144.285714,0.018434,False,76.0,8
4,36,200.0,1980-12-01,2009,2010,2013,660.0,4.0,206862,125000,...,0.0,F,180.0,180.0,False,165.000000,0.003191,False,42.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3161,26651,605.0,1982-11-01,2014,2014,2014,605.0,1.0,12505,70000,...,0.0,F,605.0,605.0,True,605.000000,0.048381,False,40.0,9
3162,26684,600.0,1940-08-01,2005,2012,2012,1200.0,2.0,62147,87500,...,0.0,M,600.0,600.0,False,600.000000,0.019309,False,82.0,9
3163,26741,5000.0,1941-08-01,2020,2021,2021,10000.0,2.0,321380,187500,...,0.0,F,5000.0,5000.0,False,5000.000000,0.031116,False,81.0,9
3164,26784,20000.0,1965-09-20,1995,2012,2020,57692.0,22.0,57692,0,...,1.0,F,40.0,250.0,False,2622.363636,1.000000,False,57.0,9
